# Spaces

In linear inferences we must be mindfull of the spaces in which we are working, especially if we work in a parameter free model case. The three spaces used are
1. $\mathcal{M}$ the model space
2. $\mathcal{D}$ the data space
3. $\mathcal{P}$ the property space

Usually $\mathcal{P}$ and $\mathcal{D}$ will be just $\mathbb{R}^N$, while $\mathcal{M}$ will be either $\mathbb{R}^N$, $L_2[\Omega]$ the space of square integrable functions defined over $\Omega$, or $PC_b[\Omega]$ the space of piece-wise continuous and bounded functinos defined over $\Omega$. In this packacge the abstract class "Space" defines such spaces. We will only look at $\mathbb{R}^N$ in this notebook.

To work with these mathematical spaces we must import the "spaces" module.

In [23]:
import sys
sys.path.append('/disks/data/PhD/BGSOLA/SOLA_DLI/core')
from core.main_classes.spaces import *

## Example 1: $\mathbb{R}$

For example, to create an object that represents the space of real numbers $\mathbb{R}$ we do:

In [24]:
R = RN(dimension=1) # Object for the space of real numbers

We can generate some random members of this space

In [25]:
print(R.random_member())

[[-80.21424363]]


Or more members at once

In [26]:
print(R.random_member(N=5))

[  0.39475941  70.37937589  38.98509227 -34.13749599 -89.00206836]


We can also add members to our space. Mathematically, all possible members already exist, so by "adding" a member, we mean creating a dictionary that holds the name and the values of that member.

In [27]:
R.add_member(member_name='first', member=1)
print(R.members)

{'first': 1}


We can also check if something is a member of our space using the ".check_if_member" method. In our case only integers or floats will be considered members

In [28]:
print(R.check_if_member(1)) # A real number - ok
print(R.check_if_member(1j)) # Imaginary number - not ok
print(R.check_if_member('s')) # String, not ok
print(R.check_if_member(np.array([np.pi]))) # np.ndarray of shape (1,) - not ok
print(R.check_if_member(np.array([np.pi, 1]))) # np.ndarray of shape (N,) N>1 - not ok
print(R.check_if_member(np.array([1j]))) # np.ndarray containng non floats/ints - not ok

True
False
False
False
False
False


Since $\mathbb{R}$ is a Hilbert space, we also have an inner product defined on it, which corresponds to simple real number multiplication:

In [29]:
print(R.inner_product(1,2))

2


By default the inner_product method will not check if the two members are actually members of the space. If they are not, you will either get some errors or bugs. You can avoid this by turning to True the "check_if_member" flag. This will make it so that an exception will be raised if one or both of the members are not actually members of the space.

In [30]:
# This should rise and exception. I keep it commented because otherwise "Run
# All" will not run any cells after this one 
# R.inner_product(1,2j, check_if_member=True) 

The inner product induces a norm, which in this case is just the absolute value:

In [31]:
print(R.norm(-5))

5.0


## Example 2: $\mathbb{R}^3$

We can also create the 3D real space:

In [32]:
R3 = RN(dimension=3)

All the methods mentioned above translate in the obvious way:

In [33]:
# Random members
print('Random member',R3.random_member())
# Add members 
R3.add_member(member_name='first', member=np.array([[np.sqrt(5)],[6],[np.pi]]))
print(R3.members)
# Checks if members or not
print(R3.check_if_member(1)) # A real number - not ok
print(R3.check_if_member(1j)) # Imaginary number - not ok
print(R3.check_if_member('s')) # String, not ok
print(R3.check_if_member(np.array([np.pi]))) # np.ndarray of shape (1,) - not ok
print(R3.check_if_member(np.array([np.pi, 1, 2]))) # np.ndarray of shape (N,) N=3 - not ok because it is a row vector
print(R3.check_if_member(np.array([[np.pi], [1], [2]]))) # np.ndarray of shape (N,) N=3 - ok because it is a column vector
print(R3.check_if_member(np.array([1j, 1, 1]))) # np.ndarray containng non floats/ints - not ok
# Inner product
member1 = np.array([[1],[1],[1]])
member2 = np.array([[5],[6],[7]])
print('Inner product', R3.inner_product(member1, member2))
# Norm (L2 norm)
print('Norm', R3.norm(member1))

Random member [[-79.58894738]
 [ 18.34699517]
 [-46.4093685 ]]
{'first': array([[2.23606798],
       [6.        ],
       [3.14159265]])}
False
False
False
False
False
True
False
Inner product 18
Norm 1.7320508075688772
